**Description**: Shows how to implement sensitivity analysis in JuMP.

**Author**: Jack Dunn

**License**: <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

## Sensitivity Analysis using JuMP

In this notebook, we will see how to use JuMP to conduct sensitivity analysis after solving a linear program and replicate the output offered by other packages (e.g. the Sensitivity Report in Excel Solver).

We will consder a production planning problem by J E Beasley, made available on [OR-Notes](http://people.brunel.ac.uk/~mastjjb/jeb/or/lpsens_solver.html). 

### Problem Statement and Model Formulation

(This section is reproduced exactly from the above link)

A company manufactures four variants of the same product and in the final part of the manufacturing process there are assembly, polishing and packing operations. For each variant the time required for these operations is shown below (in minutes) as is the profit per unit sold.

| Variant | Assembly | Polish | Pack | Profit |
| :-----: | :------: | :----: | :--: | :----: |
| 1       | 2        | 3      | 2    | 1.50   |
| 2       | 4        | 2      | 3    | 2.50   |
| 3       | 3        | 3      | 2    | 3.00   |
| 4       | 7        | 4      | 5    | 4.50   |

Given the current state of the labour force the company estimate that, each year, they have 100000 minutes of assembly time, 50000 minutes of polishing time and 60000 minutes of packing time available. How many of each variant should the company make per year and what is the associated profit?

#### Variables

Let $x_i \geq 0$ be the number of units of variant i ($i=1,2,3,4$) made per year

#### Constraints

The operation time limits give the following constraints:

- $2 x_1 + 4 x_2 + 3 x_3 + 7 x_4 \leq 100000~~~$ (assembly) 
- $3 x_1 + 2 x_2 + 3 x_3 + 4 x_4 \leq 50000~~~$ (polish) 
- $2 x_1 + 3 x_2 + 2 x_3 + 5 x_4 \leq 60000~~~$ (pack)

#### Objective

Presumably to maximise profit - hence we have

- $\max 1.5 x_1 + 2.5 x_2 + 3.0 x_3 + 4.5 x_4$

### Complete Formulation

$$\begin{align*}
\max~~~ &1.5 x_1 + 2.5 x_2 + 3.0 x_3 + 4.5 x_4 \\
\textrm{s.t.}~~~ &2 x_1 + 4 x_2 + 3 x_3 + 7 x_4 \leq 100000 \\
                 &3 x_1 + 2 x_2 + 3 x_3 + 4 x_4 \leq 50000 \\
                 &2 x_1 + 3 x_2 + 2 x_3 + 5 x_4 \leq 60000 \\
                 &x_i \geq 0,~~~i = 1, 2, 3, 4
\end{align*}$$

### Solving the model in JuMP

Now we can formulate and solve this model in JuMP:

In [264]:
# Define the data
m = collect(1:3)
n = collect(1:4)
c = Dict(
        n[1] => 1.5, 
        n[2] => 2.5, 
        n[3] => 3.0,
        n[4] => 4.5
    )
A = Dict(
        m[1] => [2, 4, 3, 7],
        m[2] => [3, 2, 3, 4],
        m[3] => [2, 3, 2, 5]
    )
b = Dict(
        m[1] => 100000,
        m[2] => 50000,
        m[3] => 60000
    )

# Import necessary packages and define model
using JuMP
using Gurobi  # We need Gurobi for Sensitivity Analysis later
model = Model(with_optimizer(Gurobi.Optimizer))

# Define the variables
@variable(model, x[n] >= 0)

# Add the objective
@objective(model, Max, sum(c[i] * x[i] for i in n))

# Add the constraints row-by-row, naming them according to each resource
# The names will allow us to refer to the constraints during sensitivity analysis
@constraint(model, con[i in m], sum(A[i][j] * x[j] for j in n) <= b[i])
# print(bombom)
# Solve the model and show the optimal solution and objective value
print(model)
optimize!(model)

for i in n
    println("x", i, ": ", JuMP.value(x[i]))
end

Academic license - for non-commercial use only
Max 1.5 x[1] + 2.5 x[2] + 3 x[3] + 4.5 x[4]
Subject to
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[3] ≥ 0.0
 x[4] ≥ 0.0
 2 x[1] + 4 x[2] + 3 x[3] + 7 x[4] ≤ 100000.0
 3 x[1] + 2 x[2] + 3 x[3] + 4 x[4] ≤ 50000.0
 2 x[1] + 3 x[2] + 2 x[3] + 5 x[4] ≤ 60000.0
Academic license - for non-commercial use only
Optimize a model with 3 rows, 4 columns and 12 nonzeros
Coefficient statistics:
  Matrix range     [2e+00, 7e+00]
  Objective range  [2e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+04, 1e+05]
Presolve time: 0.00s
Presolved: 3 rows, 4 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4000000e+31   7.875000e+30   1.400000e+01      0s
       2    5.8000000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds
Optimal objective  5.800000000e+04
x1: 0.0
x2: 16000.000000000002
x3: 5999.999999999999
x4: 0.0


We see that the optimal production plan is to make 16000 units of variant 1 and 6000 units of variant 2, with an optimal profit of \$58000

### Sensitivity Analysis

Once we have solved a model, it is often useful to analyze the sensitivity of the solution to the model parameters. Other modeling tools like Excel Solver can produce a Sensitivity Report, which summarizes all of the sensitivity information in one table. 

The Sensitivity Report produced for the production planning solution above is as follows (image from OR-Tools):

![Sensitivity Report](https://i.imgur.com/6tYVVwH.gif)

The table contains information on the shadow prices and reduced costs in the model, as well as the ranges on the cost coefficients and right-hand side values for which the current basis is optimal.

We will now reproduce this table using our JuMP model

#### Final Values

We can get the final values of the variables with `getValue()` as before:

In [265]:
x_final = []
for i in n
    append!(x_final, JuMP.value(x[i]))
end
print(x_final)

Any[0.0, 16000.0, 6000.0, 0.0]

We can get the final values of the constraints by calculating $Ax$:

In [266]:
con_final = []
for i in m
    append!(con_final, sum((A[i][j] * x_final[j]) for j in n))
end
print(con_final)

Any[82000.0, 50000.0, 60000.0]

#### Reduced Costs/Shadow Prices

We can extract the reduced costs by calling `getDual()` on the variables:

In [322]:
#print(JuMP.dual())
# JuMP.dual(model)
# for i in m
println(model)
# end
JuMP.dual(model)

Max 1.5 x[1] + 2.5 x[2] + 3 x[3] + 4.5 x[4]
Subject to
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[3] ≥ 0.0
 x[4] ≥ 0.0
 2 x[1] + 4 x[2] + 3 x[3] + 7 x[4] ≤ 100000.0
 3 x[1] + 2 x[2] + 3 x[3] + 4 x[4] ≤ 50000.0
 2 x[1] + 3 x[2] + 2 x[3] + 5 x[4] ≤ 60000.0



MethodError: MethodError: no method matching dual(::Model)
Closest candidates are:
  dual(!Matched::ConstraintRef{Model,NonlinearConstraintIndex,Shape} where Shape<:AbstractShape) at /home/lcd/.julia/packages/JuMP/jnmGG/src/nlp.jl:162
  dual(!Matched::ConstraintRef{Model,#s1044,Shape} where Shape<:AbstractShape where #s1044<:MathOptInterface.ConstraintIndex) at /home/lcd/.julia/packages/JuMP/jnmGG/src/constraints.jl:480

In [268]:
assembly = con[1]
polish = con[2]
pack = con[3]

con[3] : 2 x[1] + 3 x[2] + 2 x[3] + 5 x[4] ≤ 60000.0

JuMP 0.19 não calcula o dual


Similarly, we can extract the shadow prices by using `getDual()` on our constraint references:

In [317]:
JuMP.dual(assembly)

-0.0

In [270]:
JuMP.dual(polish)

-0.8

In [271]:
JuMP.dual(pack)

-0.3

We can put these together into a vector as well:

In [313]:
shadow_prices = JuMP.dual([assembly, polish, pack])

MethodError: MethodError: no method matching dual(::Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1})
Closest candidates are:
  dual(!Matched::ConstraintRef{Model,NonlinearConstraintIndex,Shape} where Shape<:AbstractShape) at /home/lcd/.julia/packages/JuMP/jnmGG/src/nlp.jl:162
  dual(!Matched::ConstraintRef{Model,#s1044,Shape} where Shape<:AbstractShape where #s1044<:MathOptInterface.ConstraintIndex) at /home/lcd/.julia/packages/JuMP/jnmGG/src/constraints.jl:480

#### Current Values

The "Objective Coefficient" and "Constraint R. H. Side" columns simply contain the values of $c$ and $b$, respectively:

In [314]:
obj_coeff = c

Dict{Int64,Float64} with 4 entries:
  4 => 4.5
  2 => 2.5
  3 => 3.0
  1 => 1.5

In [274]:
con_rhs = b

Dict{Int64,Int64} with 3 entries:
  2 => 50000
  3 => 60000
  1 => 100000

#### Allowable Increase/Decrease

We now want to retrieve the range of parameter values for which the optimal basis does not change. There are multiple ways to do this.

One approach is to get the optimal basis using the `MathProgBase.getbasis()` function, and this can then be used to calculate the allowable ranges of increase and decrease using standard linear programming theory of sensitivity analysis.

Alternatively, some solvers (like Gurobi) provide this information directly without the need for us to compute it manually. In this case, we can access the data through the Gurobi API directly, which is what we will do in the rest of this example. 

In [275]:
# Import Gurobi so we can access the API
import Gurobi

# Get a reference to the internal Gurobi model so we can use the API
g = getrawsolver(model)

UndefVarError: UndefVarError: getrawsolver not defined

We can now access the sensitivity information directly. To do this, we make use of the `get_dblattrarray()` function from [Gurobi.jl](https://github.com/JuliaOpt/Gurobi.jl/blob/master/src/grb_attrs.jl#L58), which allows us to retrieve [attributes from the Gurobi API](http://www.gurobi.com/documentation/6.5/refman/attributes.html) that are arrays of floating point values. 

When calling `get_dblattrarray()`, we have to specify the internal Gurobi model, the name of the attribute we want to retrieve, the index at which to starting reading from the array, and the number of values to read.

In our case, we use the `SARHSLow` and `SARHSUp` attributes to get the lower and upper bounds on the RHS values, and in each case we start at the first value and read in a total of $m$ values. Similarly, we use `SAObjLow` and `SAObjUp` to get the lower and upper bounds for the objective coefficients, and in this case we read in all $n$ values.

In [276]:
# RHS value lower and upper bounds
rhs_lb = Gurobi.get_dblattrarray(g, "SARHSLow", 1, m)
rhs_ub = Gurobi.get_dblattrarray(g, "SARHSUp", 1, m)
@show rhs_lb
@show rhs_ub

# Objective coefficient lower and upper bounds
obj_lb = Gurobi.get_dblattrarray(g, "SAObjLow", 1, n)
obj_ub = Gurobi.get_dblattrarray(g, "SAObjUp", 1, n)
@show obj_lb
@show obj_ub;

UndefVarError: UndefVarError: g not defined

The order of values in these arrays is not necessarily obvious in larger problems, and generally we do not know the order in which the information is returned by Gurobi. We can use the `getLinearIndex()` function on our variables and constraints to find their position in these arrays:

In [277]:
x_order = map(linearindex, x)

UndefVarError: UndefVarError: linearindex not defined

In [278]:
con_order = map(linearindex, [assembly, polish, pack])

UndefVarError: UndefVarError: linearindex not defined

We see that the variables and constraints are already ordered for us, but this isn't true all the time, so it pays to always rearrange the arrays according to this ordering in case the order is different

In [279]:
rhs_lb_sorted = rhs_lb[con_order];
rhs_ub_sorted = rhs_ub[con_order];
obj_lb_sorted = obj_lb[x_order];
obj_ub_sorted = obj_ub[x_order];

UndefVarError: UndefVarError: rhs_lb not defined

Now, we can use these lower and upper bounds to obtain the allowable increase and decrease on each objective coefficient and RHS value:

In [280]:
@show rhs_dec = con_rhs - rhs_lb_sorted;
@show rhs_inc = rhs_ub_sorted - con_rhs;

@show obj_dec = obj_coeff - obj_lb_sorted;
@show obj_inc = obj_ub_sorted - obj_coeff;

UndefVarError: UndefVarError: rhs_lb_sorted not defined

### Final Sensitivity Table

We can put all of this together to form the final Sensitivity Report tables.

First, the variables:

In [281]:
var_sensitivity = [x_final red_costs obj_coeff obj_inc obj_dec]

UndefVarError: UndefVarError: obj_inc not defined

And similarly for the constraints:

In [282]:
con_sensitivity = [con_final shadow_prices con_rhs rhs_inc rhs_dec]

UndefVarError: UndefVarError: shadow_prices not defined

This leads to the following tables, which we see are identical to those produced by Excel Solver:

**Variables:**

| Name  | Final Value | Red. Cost | Obj. Coeff | Allow. Inc. | Allow. Dec. |
| :---: | ----------: | --------: | ---------: | ----------: | ----------: |
| $x_1$ | 0           | -1.5      | 1.5        | 1.50        | 1E+100      |
| $x_2$ | 16000       | 0.0       | 2.5        | 2.00        | 0.1429      |
| $x_3$ | 6000        | 0.0       | 3.0        | 0.75        | 0.5         |
| $x_4$ | 0           | -0.2      | 4.5        | 0.20        | 1E+100      |

**Constraints:**

| Name     | Final Value | Shadow Price | RHS    | Allow. Inc. | Allow. Dec. |
| :------: | ----------: | -----------: | -----: | ----------: | ----------: |
| Assembly | 82000       | 0.0          | 100000 | 1E+100      | 18000       |
| Polish   | 50000       | 0.8          | 50000  | 40000       | 10000       |
| Pack     | 60000       | 0.3          | 60000  | 15000       | 26667       |
